# Using the simulator

In this notebook you will learn how to use the simulator to test out methods.

## Setting up a connection with the robot

As described in the [basic liquid handling tutorial](basic), we will use the {class}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler` class to control the robot. This time, however, instead of using the Hamilton {class}`~pylabrobot.liquid_handling.backends.hamilton.STAR` backend, we are using virtual the {class}`~pylabrobot.liquid_handling.backends.simulation.simulator_backend.SimulatorBackend` backend. This means that the liquid handling protocol will look exactly the same, but the commands are sent to the simulator instead of a real physical robot.

In [1]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends.simulation.simulator_backend import SimulatorBackend

Again, we use the {class}`~pylabrobot.resources.hamilton.STARLetDeck`:

In [2]:
from pylabrobot.resources.hamilton import STARLetDeck

In [3]:
sb = SimulatorBackend()
lh = LiquidHandler(backend=sb, deck=STARLetDeck())

Calling {func}`~pylabrobot.liquid_handling.backends.simulation.SimulatorBackend.setup` will set up the simulation server and open it in a new browser tab.

In [4]:
lh.setup()

Simulation server started at http://127.0.0.1:2121
File server started at http://127.0.0.1:1337. Open this URL in your browser.


Call {func}`~pylabrobot.liquid_handling.backends.simulation.SimulatorBackend.wait_for_connection` to wait for the simulator to connect to the server. This may take a few seconds. (Be sure to open the file server link in the browser if it doesn't open automatically!)

In [5]:
sb.wait_for_connection()

For the optimal experience, we recommend that you run the notebook and simulator side by side.

![The empty simulator](./img/simulator/empty.png)

## Assigning plates and tips

With the simulator, {func}`assigning resources <pylabrobot.resources.hamilton.STARLetDeck.assign_child_resource>` has the additional affect of placing the resources on the simulated deck. They will appear immediately.

In [6]:
from pylabrobot.resources import (
  TIP_CAR_480_A00,
  PLT_CAR_L5AC_A00,
  Cos_96_DW_1mL,
  STF_L
)

In [7]:
tip_car = TIP_CAR_480_A00(name='tip carrier')
for i in range(2):
    tip_car[i] = STF_L(name=f'tip_rack_{i}')

In [8]:
lh.deck.assign_child_resource(tip_car, rails=1, replace=True)

When you set or delete subresources in a {class}`~pylabrobot.resources.abstract.Carrier`, that will immediately be reflected in the UI.

In [9]:
tip_car[2] = STF_L(name='tip_rack_2')

In [10]:
tip_car[3] = STF_L(name='tip_rack_3')

In [11]:
del tip_car[3]

In [12]:
tip_car[4] = STF_L(name='tip_rack_4')

In [13]:
tip_car[4] = None

In [14]:
tip_car[3] = None

In [15]:
tip_car[3] = STF_L(name='tip_rack_3')

In [16]:
tip_car[4] = STF_L(name='tip_rack_4')

In [17]:
plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
for i in range(5):
    plt_car[i] = Cos_96_DW_1mL(name=f'plate_{i}')

In [18]:
lh.deck.assign_child_resource(plt_car, rails=9, replace=True)

![The simulator after the resources have been assigned](./img/simulator/assignment.png)

## Build the deck layout: placing resources

Where you would manually place the resources like tips and liquid on the deck when using a physical system, with the Simulator you can add them using code.

### Tips

Let's {func}`~pylabrobot.liquid_handling.backends.simulation.SimulatorBackend.edit_tips` the tip racks to place tips. You can also use this method to remove tips.

In [19]:
tip_rack_0 = lh.get_resource("tip_rack_0")
sb.fill_tip_rack(tip_rack_0)

In [20]:
tip_rack_1 = lh.get_resource("tip_rack_1")
sb.fill_tip_rack(tip_rack_1)

In [21]:
tip_rack_2 = lh.get_resource("tip_rack_2")
sb.edit_tips(tip_rack_2, pattern=[[True, True, False, False]*3]*8)

In [22]:
tip_rack_3 = lh.get_resource("tip_rack_3")
sb.edit_tips(tip_rack_3, pattern=[[True, False]*6]*8)

In [23]:
tip_rack_4 = lh.get_resource("tip_rack_4")
sb.edit_tips(tip_rack_4, pattern=[[True]*6 + [False]*6]*8)

### Liquids

Adding liquid to wells works similarly. You can use {func}`~pylabrobot.liquid_handling.backends.simulation.SimulatorBackend.adjust_well_volume` to adjust the volume of individual wells in each resource. Note that the opacity of the well matches the volume of the well.

In [24]:
plate_2 = lh.get_resource("plate_2")
sb.adjust_well_volume(plate_2, pattern=[[100, 800]*6]*8)

In [25]:
plate_1 = lh.get_resource("plate_1")
sb.adjust_well_volume(plate_1, pattern=[[1000]*12]*8)

In [26]:
plate_0 = lh.get_resource("plate_0")
sb.adjust_well_volume(plate_0, pattern=[[0]*12]*8)

![Simulator after the tips have been placed and the volumes adjusted](./img/simulator/resources.png)

## Liquid handling

Once the layout is complete, you can run the same commands as described in the [basic liquid handling tutorial](basic).

### Picking up tips

In [27]:
lh.pick_up_tips(tip_rack_0["A1:C1"])

In [28]:
lh.discard_tips(tip_rack_0["A1:C1"])

### Aspirating and dispensing

In [29]:
lh.pick_up_tips(tip_rack_0["A1"])

In [30]:
lh.aspirate(plate_2["A2"], vols=[300])

In [31]:
plate_4 = lh.get_resource("plate_4")
lh.dispense(plate_4["A2"], vols=[300])

In [32]:
lh.discard_tips(tip_rack_0["A1"])

### Aspirating using CoRe 96

The CoRe 96 head supports liquid handling operations for 96 channels at once. Here's how to use:

- {func}`~pylabrobot.liquid_handling.LiquidHandler.pick_up_tips_96` for picking up 96 tips;
- {func}`~pylabrobot.liquid_handling.LiquidHandler.aspirate_plate` for aspirating liquid from the deck, where the `pattern` array controls which wells are aspirated from;
- {func}`~pylabrobot.liquid_handling.LiquidHandler.dispense_plate` for dispensing liquid to the deck, where the `pattern` array controls which wells are dispensed to, and;
- {func}`~pylabrobot.liquid_handling.LiquidHandler.discard_tips96` for discarding tips to the resource.


In [33]:
lh.pick_up_tips96(tip_rack_0)

In [34]:
lh.aspirate_plate(plate_1, volume=400)

In [35]:
lh.dispense_plate(plate_0, volume=400)

In [36]:
lh.aspirate_plate(plate_1, volume=400)

In [37]:
lh.dispense_plate(plate_0, volume=400)

In [38]:
lh.discard_tips96(tip_rack_0)

![The simulator after the liquid handling operations completed](./img/simulator/after_lh.png)

## Shutting down

When you're done, remember to shut down the simulator by calling {func}`~pylabrobot.liquid_handling.LiquidHandler.stop`.

In [39]:
lh.stop()